In [3]:
import time
print(f"Last updated on {time.asctime(time.gmtime())} UTC")

Last updated on Wed Dec 25 08:31:53 2019 UTC


### Purpose

The purpose this notebook serves is to show the design process of the Back-off n-grams Language Models (BNLM) with enhancement from Neural Network Language Models (NNLM) described in the paper "Neural Network Language Model for Chinese Pinyin Input Method Engine" by Chen et all. The task at hand with this language model is given a sequence of syllables, to predict which is most likely the next syllable, a task also known as candidate sentence generation. The model is to be implemented into HKIME, an intelligent input method for Cantonese. There would be three sections in this notebook, each building upon the previous. 

### Breakdown

- Section 1: Basic n-grams using particle filtering
- Section 2: Back-off n-grams language model with interpolated Kneser-Ney smoothing
- Section 3: BNLM (from section 2) with probabilities calculated with NNLM

In [4]:
import random

# Jyutping Corpus Processing

In [7]:
#TODO: Add corpus processing
#TODO: Find better corpuses
#TODO: Look into webscraping to generate corpuses ourselves

FILENAME = "sources/lihkg_corpus"

with open(FILENAME, "r") as f:
    content = f.read()
    jyutping_corpus = content.splitlines()
print(jyutping_corpus)

['海港城於周二平安夜發生一連串警民衝突，最少兩名市民被捕，一名市民路經時被警方無理用警棍打頭而要送院。', '衝突發生過後，現場除了有市民和防暴警員外，佐敦南選區候任區議員陳梓維亦在場。', '他不單在場用手機進行直播，告知街坊海港城的最新情況，更表示希望他在場能 唔好畀佢 警方 亂咁濫用暴力 。', '', '', '陳梓維原本受邀在平安夜晚上到一間教會參與活動，但當他得知海港城內有衝突發生，便決定中途離開，趕到海港城現場；到場時，港威商場三樓已全層封閉，二樓亦有防暴警員戒備。', '同時，他亦見到該名較早前頭部受傷的市民在現場接受救護員治療。', '進入海港城前，陳梓維已在專頁 開 ；到場後，他眼見現場環境頗為混亂，遂決定繼續直播，向街坊進行現場報道。', '', '', '訪問途中，陳梓維提到一個小插曲，指他到場後，有一名防暴警員認得他為候任區議員，要求他 控制、處理現場環境 。', '陳梓維表示，警員當時的一番說話 即時引起全場哄動 。', '最後，該名警員在現場市民的叫嚷聲中，退至商場出入口處。', '作為候任區議員，陳梓維認為自己在場，可以 唔好畀佢 警方 亂咁濫用暴力 ，並且表示即使警方要進行拘捕，但都一定不可以使用過份武力。', '', '', '最後，記者問陳梓維在平安夜有甚麼說話想對香港人說？', '陳梓維不禁一笑，並表示希望每位香港人 平安夜能夠安全回家 ，並且能開心快樂地度過這個節日。', '', '', '當訪問完結後，又有警員走前並手指陳梓維說，這就是 會考零分都做到區議員嗰個人 。', '諷刺的是，那位警員在揶揄陳梓維的會考分數，卻不知道自己錯把陳梓維名字中的 梓  正確讀音 趾 誤讀為 辛 ，令在進行手機直播中的陳梓維，都忍不住當場糾正，令對方尷尬非常。 ']


### Set n-grams character count (n-1 in n-grams)

In [8]:
CHARACTER_COUNT = 2

# Section 1: Basic n-grams prediction model

Post-processing of the Cantonese corpus would get us a list of strings, where each string could be a phrase, a sentence, or a paragraph. For conciseness, we would call all of these sentences. In this section, we would divide up each sentence into the n-grams and then store the possible next letters for each n-gram in a python dictionary. The naive prediction algorithm would randomly pick from the possible next letters given a certain n-gram to generate candidate sentences.

### Generating n-grams dictionary

This would generate a dictionary where each key is an n-gram and the value would be a list of possible next characters.

In [9]:
#returns dictionary for prediction
def generate_n_grams_dict(processed_corpus):
    result = dict()
    for sentence in processed_corpus:
        #i is the start index of the slice
        for i in range(len(sentence) - CHARACTER_COUNT - 1): # -1 since last slice does not have next char
            grams = sentence[i:i+CHARACTER_COUNT]
            next_char = sentence[i+CHARACTER_COUNT]
            if grams in result:
                result[grams].append(next_char)
            else:
                # as long as there is an n-gram key in the dict, there would be at least one next char
                result[grams] = [next_char] 
    return result

### TODO: visualization of n-grams dict

### Prediction Model

This naive prediction model would, if the sentence has an n-gram in the dictionary, randomly select a next character from the list of potential next characters.

In [10]:
# returns a next character given an n_grams_dict and a sentence.
def predict_next_char(n_grams_dict, sentence):
    potentials = n_grams_dict.get(sentence[-CHARACTER_COUNT:], None)
    return random.choice(potentials) if potentials != None else None

### Testing

Here we would test the implementation of the naive n-grams prediction model, for comparison with more sophisticated language models. We would do two tests, the first one would generate a 200 character sentence, and the second would test the implementation analytically by seeing how many next characters it will predict correctly on the test dataset.

TODO: Add a validation dataset. The current corpus is too small to be used both for training and validation.

#### Test 1

In [11]:
def testing(sentence):
    n_grams_dict = generate_n_grams_dict(jyutping_corpus)
    tmp = sentence
    # Generate a sentence of up to 200 characters, will break if an n-gram not found in n-grams dict.
    for i in range(200):
        res = predict_next_char(n_grams_dict, tmp)
        if res == None:
            break
        else:
            tmp = tmp + res
    return tmp

# Observe that due to the stochastic nature of particle filtering, the result is not pre-determinable
print("Trial 1: ")
print(testing("最後"))
print("Trial 2: ")
print(testing("最後"))

Trial 1: 
最後，記者問陳梓維，都忍不住當場糾正，令在進行手機直播，告知街坊海港城於周二平安夜能夠安全回家 ，令在進行手機直播，向街坊海港城的最新情況，更表示，警員戒備
Trial 2: 
最後，又有警員走前並手指陳梓維已在專頁 開 ；到場後，現場除了有市民被捕，一名防暴警員在揶揄陳梓維在平安夜有甚麼說話想對香港人說


#### Test 2
**WARNING** Not a validation dataset

In [12]:
n_grams_dict = generate_n_grams_dict(jyutping_corpus)
count = 0
correct = 0
for sentence in jyutping_corpus:
    for i in range(len(sentence) - CHARACTER_COUNT - 1):
        if predict_next_char(n_grams_dict, sentence[:i+CHARACTER_COUNT]) == sentence[i+CHARACTER_COUNT]:
            correct += 1
        count += 1

print(f"Total of {count} predictions made")
print(f"{correct} predictions correct")
print(f"Prediction accuracy: {(correct/(count)) * 100}%")

Total of 629 predictions made
543 predictions correct
Prediction accuracy: 86.3275039745628%


## Section 2: Back-off n-grams language model with interpolated Kneser-Ney smoothing

### Generating n-grams dictionary

To use n-grams with a back-off model, we would need to store more information in the dictionary. In particular, we need to store not only n-grams but also everything down to a unigram. The backoff model works by testing if there is a n-gram and then backing off to n-1, n-2 and so on.

In [13]:
def generate_backoff_n_grams_dict(processed_corpus):
    result = dict()
    for cc in range(1, CHARACTER_COUNT+1):
        for sentence in processed_corpus:
            #i is the start index of the slice
            for i in range(len(sentence) - cc - 1): #-1 since last slice does not have next char
                grams = sentence[i:i+cc]
                next_char = sentence[i+cc]
                if grams in result:
                    if next_char in result[grams]:
                        result[grams][next_char] += 1
                    else:
                        result[grams][next_char] = 1
                else:
                    #as long as there is an n-gram key in the dict, there would be at least one next char
                    result[grams] = {}
                    result[grams][next_char] = 1
                    
    return result
    

In [14]:
generate_backoff_n_grams_dict(jyutping_corpus)

{'海': {'港': 5},
 '港': {'城': 5, '威': 1, '人': 2},
 '城': {'於': 1, '的': 1, '內': 1, '現': 1, '前': 1},
 '於': {'周': 1},
 '周': {'二': 1},
 '二': {'平': 1, '樓': 1},
 '平': {'安': 4},
 '安': {'夜': 4, '全': 1},
 '夜': {'發': 1, '晚': 1, '有': 1, '能': 1},
 '發': {'生': 3},
 '生': {'一': 1, '過': 1, '，': 1},
 '一': {'連': 1, '名': 2, '間': 1, '個': 1, '番': 1, '定': 1, '笑': 1},
 '連': {'串': 1},
 '串': {'警': 1},
 '警': {'民': 1, '方': 4, '棍': 1, '員': 7},
 '民': {'衝': 1, '被': 1, '路': 1, '和': 1, '在': 1, '的': 1},
 '衝': {'突': 3},
 '突': {'，': 1, '發': 2},
 '，': {'最': 1,
  '一': 1,
  '現': 1,
  '佐': 1,
  '告': 1,
  '更': 1,
  '但': 2,
  '便': 1,
  '趕': 1,
  '港': 1,
  '二': 1,
  '他': 2,
  '陳': 3,
  '遂': 1,
  '向': 1,
  '指': 1,
  '有': 1,
  '要': 1,
  '警': 1,
  '該': 1,
  '退': 1,
  '可': 1,
  '並': 3,
  '記': 1,
  '又': 1,
  '這': 1,
  '那': 1,
  '卻': 1,
  '令': 2,
  '都': 1},
 '最': {'少': 1, '新': 1, '後': 2},
 '少': {'兩': 1},
 '兩': {'名': 1},
 '名': {'市': 2, '較': 1, '防': 1, '警': 1, '字': 1},
 '市': {'民': 5},
 '被': {'捕': 1, '警': 1},
 '捕': {'，': 2},
 '路': {'經': 1}

### Prediction Model

The prediction model would first pull out the n-grams from the sentence and try to find a next word in the hash table. If the word is not found, then it would be moved to a lower n-gram until it reaches a unigram.

In [23]:
def predict_next_char_backoff(n_grams_dict, sentence):
    for cc in range(min(len(sentence), CHARACTER_COUNT), 0, -1):
        potentials = n_grams_dict.get(sentence[-cc:], None)
        if potentials != None:
            break
    return max(potentials.keys(), key=lambda x: potentials[x]) if potentials != None else None

### Testing

This would be the same testing as with Section 1. We keep the same tests to see if there is an improvement.

#### Test 1

In [26]:
def testing(sentence):
    n_grams_dict = generate_backoff_n_grams_dict(jyutping_corpus)
    tmp = sentence
    #Generate a sentence of up to 200 characters, will break if an n-gram not found in n-grams dict.
    for i in range(10):
        res = predict_next_char_backoff(n_grams_dict, tmp)
        if res == None:
            break
        else:
            tmp = tmp + res
    return tmp

print("Trial 1: ")
print(testing("最後"))

Trial 1: 
最後，現場環境頗為混亂，


Test 2

In [13]:
n_grams_dict = generate_n_grams_dict(jyutping_corpus)
count = 0
correct = 0
for sentence in jyutping_corpus:
    for i in range(len(sentence) - CHARACTER_COUNT - 1):
        if predict_next_char_backoff(n_grams_dict, sentence[:i+CHARACTER_COUNT]) == sentence[i+CHARACTER_COUNT]:
            correct += 1
        count += 1

print(f"Total of {count} predictions made")
print(f"{correct} predictions correct")
print(f"Prediction accuracy: {(correct/(count)) * 100}%")

Total of 629 predictions made
553 predictions correct
Prediction accuracy: 87.91732909379968%
